In [1]:
import os
import sys
import argparse
import json
import numpy as np
import re
import random
import logging
import _pickle as cPickle
import time
from utils import tokenize

sys.path.append('coco/PythonAPI')
print(sys.path)
from pycocotools.coco import COCO
from coco.coco_caption_py3.pycocoevalcap.eval import COCOEvalCap

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

['', '/anaconda3/lib/python36.zip', '/anaconda3/lib/python3.6', '/anaconda3/lib/python3.6/lib-dynload', '/anaconda3/lib/python3.6/site-packages', '/anaconda3/lib/python3.6/site-packages/aeosa', '/anaconda3/lib/python3.6/site-packages/IPython/extensions', '/Users/junjiedong/.ipython', 'coco/PythonAPI']


In [2]:
# set up file names and pathes
annFile='./coco/annotations/captions_val2014.json'
resFile = './coco/coco_caption_py3/results/captions_val2014_fakecap_results.json'

In [3]:
# create coco object and cocoRes object
coco = COCO(annFile)
cocoRes = coco.loadRes(resFile)

loading annotations into memory...
Done (t=0.36s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!


In [4]:
# create cocoEval object by taking coco and cocoRes
cocoEval = COCOEvalCap(coco, cocoRes)

# evaluate on a subset of images by setting
# cocoEval.params['image_id'] = cocoRes.getImgIds()
# please remove this line when evaluating the full validation set
cocoEval.params['image_id'] = cocoRes.getImgIds()

# evaluate results
tic = time.time()
cocoEval.evaluate()
print("\nEvaluation took {} seconds.\n".format(time.time() - tic))

# print output evaluation scores
for metric, score in cocoEval.eval.items():
    print("{}: {}".format(metric, score))

tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 9893, 'reflen': 9855, 'guess': [9893, 8893, 7893, 6893], 'correct': [5732, 2510, 1043, 423]}
ratio: 1.003855910705124
Bleu_1: 0.579
Bleu_2: 0.404
Bleu_3: 0.279
Bleu_4: 0.191
computing METEOR score...
METEOR: 0.195
computing Rouge score...
ROUGE_L: 0.396
computing CIDEr score...
CIDEr: 0.600

Evaluation took 11.559043169021606 seconds.

Bleu_1: 0.5793995754573356
Bleu_2: 0.40439129741018104
Bleu_3: 0.2785363856619147
Bleu_4: 0.1908290437674253
METEOR: 0.19525467177780284
ROUGE_L: 0.39625269357570847
CIDEr: 0.5997905185184199


Evaluating 1000 examples on all four metrics takes 12s -> Evaluating on our size-5000 val set will take about 1min. Good!

In [5]:
print(len(cocoEval.params['image_id']))
print(cocoEval.params['image_id'][:10])

1000
[391895, 522418, 184613, 318219, 554625, 397133, 574769, 60623, 309022, 5802]


In [6]:
fakeres_raw = json.load(open(resFile, 'r'))

In [7]:
print(len(fakeres_raw))
print(fakeres_raw[:10])

1000
[{'image_id': 404464, 'caption': 'black and white photo of a man standing in front of a building'}, {'image_id': 380932, 'caption': 'group of people are on the side of a snowy field'}, {'image_id': 565778, 'caption': 'train traveling down a train station'}, {'image_id': 431573, 'caption': 'red fire hydrant sitting on a park bench in front of a road'}, {'image_id': 322226, 'caption': 'black and white cat is sitting on top of a wooden bench'}, {'image_id': 237669, 'caption': 'baseball player swinging a bat at a game'}, {'image_id': 351053, 'caption': 'laptop computer sitting on top of a table'}, {'image_id': 344860, 'caption': 'zebra standing on top of a lush green field'}, {'image_id': 40102, 'caption': 'group of giraffes standing next to each other in a grassy field'}, {'image_id': 95427, 'caption': 'close up of a pile of oranges sitting on a table'}]


In [8]:
print(len(set([res['image_id'] for res in fakeres_raw])))

1000
